In [1]:
!pip install facenet_pytorch

In [2]:
!pip install matplotlib

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob
import os
# from tensorflow.keras.models import load_model

# from tensorflow.keras.models import Model
import os
import warnings


from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim



/home/csgrad/kaushik3/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

In [5]:
# tempIter = iter(testloader)
# images,gen_pairs, imp_pairs = next(tempIter)
# # imshow(images[0])
# # print(labels['age'][0],labels['gender'][0],labels['ethnicity'][0])
# # print(image_pairs)
# imshow(images[0])
# imshow(gen_pairs[0][0])
# imshow(imp_pairs[0][0])

In [6]:
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=FutureWarning) 
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print('Running on device: {}'.format(device))

Running on device: cuda:1


In [7]:
file_path = '/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Code/FHE/MRL+PP+FHE_FaceNet_BFW.txt'
# Open the file and read its lines into a list of strings without newline characters
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file]

# Display the list of strings
print(len(lines))
print(lines[0])


20000
...


In [8]:
print(len(lines[0]))
print(len(lines[1]))

30827
30826


In [9]:
max_length = max(lines, key=len)
print(len(max_length))

30841


In [10]:
padded_lines = [s.ljust(len(max_length), '0') for s in lines]

In [11]:
len(padded_lines)

20000

In [12]:
def ageLabel(age):

    # Define class ranges
    class_ranges = [(15, 22),(22,40),(40,60),(60,80)]

# Check if both values fall into the same class range
    if(class_ranges[0][0] <= age and age < class_ranges[0][1]):
        return 0
    elif(class_ranges[1][0] <= age and age < class_ranges[1][1]):
        return 1
    elif(class_ranges[2][0] <= age and age < class_ranges[2][1]):
        return 2
    elif(class_ranges[3][0] <= age and age < class_ranges[3][1]):
        return 3
    else:
        return 0
    
def genderMapping(gender):
    if gender == 'males':
        return 0
    elif gender == 'females':
        return 1
    return -1

def ethMapping(eth):
    if eth == 'indian':
        return 0
    elif eth == 'white':
        return 1
    elif eth == 'asian':
        return 2
    elif eth == 'black':
        return 3
    return -1


In [13]:
data = pd.read_csv('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Data/BFW/bfw_id.csv')

In [14]:

age_list = []
gender_list = []
ethn_list =[]
id_list = []


for i in range(len(data)):
    row = data.iloc[i]
    id_list.append(row['Id'])
    gender_list.append(genderMapping(row['Gender']))
    ethn_list.append(ethMapping(row['Ethnicity']))
    age_list.append(row['predictedAge'])

In [15]:
print("Age length = ", len(age_list))
print("Eth length = ", len(ethn_list))
print("Gender length = ", len(gender_list))
print("Id length = ", len(id_list))

Age length =  20000
Eth length =  20000
Gender length =  20000
Id length =  20000


In [16]:
!pip install scikit-learn

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Age SVM

In [18]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, age_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.82175


#Gender SVM

In [19]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, gender_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.495


#Id SVM

In [20]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, id_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.0


#Eth SVM

In [21]:


# Convert strings to feature vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(padded_lines)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, ethn_list, test_size=0.2, random_state=42)

# Train a Multiclass SVM classifier
svm_classifier = SVC(kernel='poly', C=1.0)
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.24
